In [5]:
# 데이터 로드 & 확인
import pandas as pd
import json
import os

# 원본 파일 읽기
df_main = pd.read_csv('../data/raw/농기계보유현황.csv', encoding='cp949')
df_gyeongbuk = pd.read_csv('../data/raw/농기계보유현황_경북.csv', encoding='cp949')
df_jeonnam = pd.read_csv('../data/raw/농기계보유현황_전남.csv', encoding='cp949')
df_jeonbuk = pd.read_csv('../data/raw/농기계보유현황_전북.csv', encoding='cp949')

print("메인 파일:", df_main.shape)
print("경북 파일:", df_gyeongbuk.shape)
print("전남 파일:", df_jeonnam.shape)
print("전북 파일:", df_jeonbuk.shape)
print(df_main.head())

메인 파일: (2668, 6)
경북 파일: (22, 19)
전남 파일: (22, 22)
전북 파일: (14, 15)
     연도     시도  시군구     기종   기종상세  보유현황
0  2018  서울특별시  노원구  동력경운기  동력경운기     2
1  2018  서울특별시  양천구  동력경운기  동력경운기     7
2  2018  서울특별시  강서구  동력경운기  동력경운기    48
3  2018  서울특별시  서초구  동력경운기  동력경운기    13
4  2018  서울특별시  강남구  동력경운기  동력경운기    10


In [6]:
# 지역명 변환 규칙
sido_mapping = {
    '전라남도': '전남', '전북특별자치도': '전북', '전라북도': '전북',
    '강원특별자치도': '강원', '강원도': '강원', '경기도': '경기',
    '경상남도': '경남', '경상북도': '경북', '충청남도': '충남', '충청북도': '충북'
}

metropolitan_cities = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', 
                      '광주광역시', '대전광역시', '울산광역시']

def convert_region_name(sido, sigungu):
    if sido in metropolitan_cities:
        return sido
    elif sido == '세종특별자치시':
        return '세종시'
    elif sido == '제주특별자치도':
        return '제주시' if '제주시' in sigungu else '서귀포시'
    else:
        short_sido = sido_mapping.get(sido, sido)
        # 시 단위 구 통합
        if '구' in sigungu:
            city_name = sigungu.split(' ')[0] if ' ' in sigungu else sigungu.replace('구', '시')
            return f"{short_sido} {city_name}"
        else:
            return f"{short_sido} {sigungu}"

print("변환 규칙 설정 완료")

변환 규칙 설정 완료


In [7]:
# 데이터 집계
all_results = []

# 1. 메인 파일 처리 (기존 방식)
if '시도' in df_main.columns and '시군구' in df_main.columns and '보유현황' in df_main.columns:
    df_main['지역'] = df_main.apply(lambda row: convert_region_name(row['시도'], row['시군구']), axis=1)
    main_result = df_main.groupby('지역')['보유현황'].sum().reset_index()
    main_result.columns = ['지역', '농기계보유현황']
    all_results.append(main_result)
    print("메인 파일 처리 완료")

# 2. 경북 파일 처리 (시군구별)
if len(df_gyeongbuk) > 0:
    gyeongbuk_results = []
    first_col = df_gyeongbuk.columns[0]  # 시군구 컬럼
    numeric_cols = df_gyeongbuk.select_dtypes(include=['int64', 'float64']).columns
    
    for _, row in df_gyeongbuk.iterrows():
        sigungu = row[first_col]
        total_count = row[numeric_cols].sum()
        gyeongbuk_results.append({
            '지역': f'경북 {sigungu}',
            '농기계보유현황': int(total_count)
        })
    
    gyeongbuk_df = pd.DataFrame(gyeongbuk_results)
    all_results.append(gyeongbuk_df)
    print(f"경북 파일 처리 완료: {len(gyeongbuk_df)}개 시군구")

# 3. 전남 파일 처리 (시군구별)
if len(df_jeonnam) > 0:
    jeonnam_results = []
    first_col = df_jeonnam.columns[0]  # 시군구 컬럼
    numeric_cols = df_jeonnam.select_dtypes(include=['int64', 'float64']).columns
    
    for _, row in df_jeonnam.iterrows():
        sigungu = row[first_col]
        total_count = row[numeric_cols].sum()
        jeonnam_results.append({
            '지역': f'전남 {sigungu}',
            '농기계보유현황': int(total_count)
        })
    
    jeonnam_df = pd.DataFrame(jeonnam_results)
    all_results.append(jeonnam_df)
    print(f"전남 파일 처리 완료: {len(jeonnam_df)}개 시군구")

# 4. 전북 파일 처리 (시군구별)
if len(df_jeonbuk) > 0:
    jeonbuk_results = []
    first_col = df_jeonbuk.columns[0]  # 시군구 컬럼
    numeric_cols = df_jeonbuk.select_dtypes(include=['int64', 'float64']).columns
    
    for _, row in df_jeonbuk.iterrows():
        sigungu = row[first_col]
        total_count = row[numeric_cols].sum()
        jeonbuk_results.append({
            '지역': f'전북 {sigungu}',
            '농기계보유현황': int(total_count)
        })
    
    jeonbuk_df = pd.DataFrame(jeonbuk_results)
    all_results.append(jeonbuk_df)
    print(f"전북 파일 처리 완료: {len(jeonbuk_df)}개 시군구")

# 5. 모든 결과 통합
result_df = pd.concat(all_results, ignore_index=True)
result_df = result_df.sort_values('농기계보유현황', ascending=False)

print(f"\n총 {len(result_df)}개 지역 데이터")
print(result_df.head(10))

메인 파일 처리 완료
경북 파일 처리 완료: 22개 시군구
전남 파일 처리 완료: 22개 시군구
전북 파일 처리 완료: 14개 시군구

총 77개 지역 데이터
        지역  농기계보유현황
18   인천광역시    72077
12   대구광역시    48582
10  경기 평택시    46269
17   울산광역시    43741
26  경북 상주시    38150
29  경북 의성군    37591
22  경북 안동시    34715
16     세종시    31840
20  경북 경주시    30015
21  경북 김천시    29853


In [8]:
# 결과 저장
os.makedirs('../docs/data', exist_ok=True)
os.makedirs('../data', exist_ok=True)

with open('../docs/data/농기계보유현황.json', 'w', encoding='utf-8') as f:
    json.dump(result_df.to_dict('records'), f, ensure_ascii=False, indent=2)

result_df.to_csv('../data/농기계보유현황.csv', index=False, encoding='utf-8-sig')

print("✓ 파일 저장 완료")
print(f"총 {len(result_df)}개 지역 데이터")
print(f"총 농기계: {result_df['농기계보유현황'].sum():,}대")

✓ 파일 저장 완료
총 77개 지역 데이터
총 농기계: 1,232,726대
